In [28]:
%load_ext autoreload
%autoreload 2

import sys, os
import pickle

import numpy as np
import matplotlib.pyplot as plt 
import torch
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
import pandas as pd
import networkx as nx
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm

from dpp.data.network import PPINetwork
from dpp.util import Params, prepare_sns, load_mapping
from dpp.data.associations import load_diseases

os.chdir("/Users/sabrieyuboglu/Documents/School/SNAP/projects/disease-protein-prediction")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
diseases = load_diseases("data/associations/disgenet-associations.csv")
entrez_to_name = load_mapping("data/protein/symbol_to_entrez.txt", "\t", reverse=True,
                              b_transform=int)

In [6]:
network = PPINetwork("data/networks/bio-pathways-network.txt")

# Common Interactors

In [23]:
def compute_common_interactors(diseases, network, null=False): 
    """
    """
    adj = network.adj_matrix
    ci_pairs = [set() for i in range(len(network))]
    ci_diseases = [set() for i in range(len(network))]
    for disease in tqdm(diseases.values()):
        proteins = disease.to_node_array(network)
        if null:
            proteins = np.random.choice(adj.shape[0], len(proteins), replace=False)

        for a, protein_a in enumerate(proteins):
            for b, protein_b in enumerate(proteins):
                if a < b:
                    break 
                neighbors_a = adj[protein_a, :]
                neighbors_b = adj[protein_b, :]
                cis = np.where(np.multiply(neighbors_a, neighbors_b) == 1)[0]
                for ci in cis:
                    ci_pairs[ci].add((protein_a, protein_b))
                    ci_diseases[ci].add(disease.id)
    return ci_pairs, ci_diseases

In [24]:
null_pairs = []
null_diseases = []
for i in range(30): 
    ci_pairs, ci_diseases  = compute_common_interactors(diseases, network, null=True)
    null_pairs.append(ci_pairs)
    null_diseases.append(ci_diseases)

100%|██████████| 2297/2297 [04:01<00:00,  9.53it/s]  


In [25]:
null_n_pairs = [np.array(map(len, pairs)) for pairs in null_pairs]

In [ ]:
with open("experiments/dpp_evaluate/lci/null_pairs.pkl",'wb') as f:
    pickle.dump(null_pairs, f)

In [ ]:
np.stack(null_n_pairs, axis=1).shape

In [ ]:
mean_null_n_pairs = np.mean(np.stack(null_n_pairs, axis=1), axis=1)

In [ ]:
degree_vec = np.array([network.nx.degree(node) for node in range(len(network))])